In [1]:
import env
import pandas as pd
import numpy as np
import utilities as utils

In [2]:
fake_df = pd.read_csv(env.data_path + "Fake.csv")

In [3]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
fake_df.subject.unique()

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east'], dtype=object)

In [5]:
true_df = pd.read_csv(env.data_path + "True.csv")

In [6]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
true_df.subject.unique()

array(['politicsNews', 'worldnews'], dtype=object)

In [8]:
fake_df['is_fake'] = True

In [9]:
fake_df.head()

,title,text,subject,date,is_fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",True
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",True
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",True
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",True
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",True


In [10]:
true_df['is_fake'] = False

In [11]:
true_df.head()

,title,text,subject,date,is_fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",False
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",False
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",False
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",False
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",False


In [12]:
articles_df = pd.concat([true_df, fake_df], axis=0)

In [13]:
fake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
 4   is_fake  23481 non-null  bool  
dtypes: bool(1), object(4)
memory usage: 756.8+ KB


In [14]:
true_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
 4   is_fake  21417 non-null  bool  
dtypes: bool(1), object(4)
memory usage: 690.3+ KB


In [15]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   is_fake  44898 non-null  bool  
dtypes: bool(1), object(4)
memory usage: 1.8+ MB


In [16]:
articles_df = articles_df.reset_index()

In [17]:
articles_df = articles_df.drop(columns=['index'])

In [18]:
def truncate_month(string_date):
    three_letter_month = string_date[0:3]
    first_space_index = string_date.find(" ")
    day_year = string_date[first_space_index:]
    
    return three_letter_month + day_year

In [19]:
def value_in_date_format(string):
    return (len(string) == 11) | (len(string) == 12)


In [20]:
articles_df.date = articles_df.date.str.strip()

In [21]:
articles_df.date = articles_df.date.apply(truncate_month)

In [22]:
articles_df['correct_date_format'] = articles_df.date.apply(value_in_date_format)

In [35]:
for row in articles_df[articles_df.correct_date_format == False].index:
    articles_df.iloc[row].date

TypeError: replace expected at least 2 arguments, got 1

In [40]:
articles_df.iloc[30467].date.apply(lambda x : x = "Aug 16, 2016")

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (<ipython-input-40-823d01276281>, line 1)

In [38]:
articles_df.iloc[30467].date

'19-8'

In [24]:
len(articles_df.iloc[0].date)

12

In [26]:
articles_df = articles_df.drop(index=[30775, 36924, 36925, 37256, 37257, 38849, 38850, 40350, 43286, 43287])

KeyError: '[30775 36924 36925 37256 37257 38849 38850 40350 43286 43287] not found in axis'

In [32]:
articles_df[articles_df.correct_date_format == False]

,title,text,subject,date,is_fake,correct_date_format
30467,Democrat Senator Warns Mueller Not To Release ...,"According to The Hill, Democrat Senator Bob Ca...",politics,19-8,True,False
30468,MSNBC ANCHOR Flabbergasted at What Texas Teach...,If we protect every other government building ...,politics,19-8,True,False
30469,WATCH: SNOWFLAKES ASKED Communist Party Platfo...,Ami Horowitz is fantastic! Check out this man ...,politics,19-8,True,False
30470,JUST IN: BADASS GENERAL JOHN KELLY Shoved Chin...,Just one more reminder of why President Trump ...,politics,18-8,True,False
30471,DOJ’s JEFF SESSIONS Opens Investigation Into W...,Thank goodnesss Jeff Sessions is moving on fin...,politics,18-8,True,False
30472,WATCH: ’’Carpetbagger” Mitt Romney Takes Swipe...,The GOP presidential candidate who couldn t se...,politics,18-8,True,False
30473,GOTCHA! A Stammering John Podesta Stunned by a...,John Podesta is Hillary Clinton s former campa...,politics,18-8,True,False
30474,HIGH SCHOOL TEACHER Makes “Shooting At Trump” ...,EAG News A Wyoming high school is under fire ...,politics,18-8,True,False
30475,IT BEGINS…RINO MEGA-DONOR Threatens Jeb Bush: ...,A longtime Republican Party donor based in Flo...,politics,18-8,True,False
30476,BUSTED! Maxine Waters’ Ties to Black Nationali...,We reported that Maxine Waters and members of ...,politics,18-8,True,False


In [62]:
pd.to_datetime(articles_df.date, format="%b %d, %Y")

AttributeError: 'str' object has no attribute 'date'

In [21]:
articles_df.to_csv("news_articles.csv")

In [80]:
articles_df.text = articles_df.text.apply(utils.nlp_basic_clean)

In [81]:
articles_df.title = articles_df.title.apply(utils.nlp_basic_clean)

In [82]:
articles_df.text = articles_df.text.apply(utils.nlp_tokenize)

In [83]:
articles_df.title = articles_df.title.apply(utils.nlp_tokenize)

In [84]:
articles_df.text = articles_df.text.apply(utils.nlp_remove_stopwords)

In [85]:
articles_df.title = articles_df.title.apply(utils.nlp_remove_stopwords)

In [86]:
articles_df.text = articles_df.text.apply(utils.nlp_lemmatize)

In [87]:
articles_df.title = articles_df.title.apply(utils.nlp_lemmatize)

In [88]:
articles_df.to_csv("all_articles.csv")

In [92]:
positions = utils.nan_null_empty_check(articles_df)
drop_rows = list(positions['empty_positions'][0])

NaN values
Empty DataFrame
Columns: [rows, columns]
Index: []
--------------------------------
Empty values
      rows  columns
0     8970        1
1    32340        1
2    32458        1
3    32607        1
4    32642        1
..     ...      ...
627  43233        1
628  43243        1
629  43244        1
630  43274        1
631  43290        1

[632 rows x 2 columns]
--------------------------------


In [93]:
articles_with_text_df = articles_df.drop(index=drop_rows)

In [94]:
utils.nan_null_empty_check(articles_with_text_df)

NaN values
Empty DataFrame
Columns: [rows, columns]
Index: []
--------------------------------
Empty values
Empty DataFrame
Columns: [rows, columns]
Index: []
--------------------------------


{'nan_positions': (array([], dtype=int64), array([], dtype=int64)),
 'empty_positions': (array([], dtype=int64), array([], dtype=int64))}

In [95]:
articles_with_text_df.to_csv("articles_with_text.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2e382e51-55a5-40b6-ab11-4433152b5cab' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>